# 主程序 0：标准交换测试方法得到的保真度及线路参数

In [138]:
import numpy as np
from scipy.sparse import csr_matrix, coo_matrix
from scipy.optimize import minimize

from mindquantum import *

gamma = 0.01

real_circ = Circuit()
real_circ += RY(0.035).on(0)
real_circ += AmplitudeDampingChannel(gamma).on(0)
real_circ += RZ(2.861).on(0)
real_circ += AmplitudeDampingChannel(gamma).on(0)
real_circ += RY(0.606).on(0)
real_circ += AmplitudeDampingChannel(gamma).on(0)
real_circ += RY(0.361).on(1)
real_circ += AmplitudeDampingChannel(gamma).on(1)
real_circ += RZ(6.174).on(1)
real_circ += AmplitudeDampingChannel(gamma).on(1)
real_circ += RY(4.513).on(1)
real_circ += AmplitudeDampingChannel(gamma).on(1)
real_circ += X.on(1,0)
real_circ += AmplitudeDampingChannel(gamma).on(0)
real_circ += AmplitudeDampingChannel(gamma).on(1)

gen_circ = Circuit()
gen_circ += RY('theta_0').on(2)
gen_circ += AmplitudeDampingChannel(gamma).on(2)
gen_circ += RZ('theta_1').on(2)
gen_circ += AmplitudeDampingChannel(gamma).on(2)
gen_circ += RY('theta_2').on(2)
gen_circ += AmplitudeDampingChannel(gamma).on(2)
gen_circ += RY('theta_3').on(3)
gen_circ += AmplitudeDampingChannel(gamma).on(3)
gen_circ += RZ('theta_4').on(3)
gen_circ += AmplitudeDampingChannel(gamma).on(3)
gen_circ += RY('theta_5').on(3)
gen_circ += AmplitudeDampingChannel(gamma).on(3)
gen_circ += X.on(3,2)
gen_circ += AmplitudeDampingChannel(gamma).on(3)
gen_circ += AmplitudeDampingChannel(gamma).on(2)


disc_circ = Circuit()
disc_circ += X.on(2, 0)
disc_circ += AmplitudeDampingChannel(gamma).on(0)
disc_circ += AmplitudeDampingChannel(gamma).on(2)
disc_circ += X.on(3, 1)
disc_circ += AmplitudeDampingChannel(gamma).on(1)
disc_circ += AmplitudeDampingChannel(gamma).on(3)
disc_circ += H.on(0)
disc_circ += AmplitudeDampingChannel(gamma).on(0)
disc_circ += H.on(1)
disc_circ += AmplitudeDampingChannel(gamma).on(1)

ansatz = sum([real_circ, gen_circ, disc_circ])

index = [int(i, 2) for i in ['0111', '1011', '1101', '1110']]
m_coo = [coo_matrix(([1], ([i], [i])), shape=(1 << 4, 1 << 4)) for i in index] # << 移位运算
hams = [Hamiltonian(m.tocsr()) for m in m_coo]
sim = Simulator('mqvector', 4)

def fun(x):
    sim.reset()
    sim.apply_circuit(ansatz, x)
    f = np.sum([sim.get_expectation(ham).real for ham in hams])
    return f

init = [1 for i in range(len(ansatz.params_name))]
res = minimize(fun, init, method='nelder-mead')
x = res.x

gen_circ = Circuit()
gen_circ += RY('theta_0').on(0)
gen_circ += RZ('theta_1').on(0)
gen_circ += RY('theta_2').on(0)
gen_circ += RY('theta_3').on(1)
gen_circ += RZ('theta_4').on(1)
gen_circ += RY('theta_5').on(1)
gen_circ += X.on(1,0)

state = gen_circ.get_qs(pr=dict(zip(gen_circ.params_name, x)))
state = np.mat(state).reshape((4,1))

real_circ = Circuit()
real_circ += RY(0.035).on(0)
real_circ += RZ(2.861).on(0)
real_circ += RY(0.606).on(0)
real_circ += RY(0.361).on(1)
real_circ += RZ(6.174).on(1)
real_circ += RY(4.513).on(1)
real_circ += X.on(1,0)

target_state = real_circ.get_qs()
target_state = np.mat(target_state.reshape((4,1)))
fid = (np.abs(target_state.H@state)**2)[0,0].real

print('fid:', fid)
print('paras:', x)

fid: 0.9999841390578427
paras: [-0.21003175 -0.04623173  0.77686948  2.5473932   3.20918606  1.13139806]


另一种方法实现上述功能

In [98]:
import numpy as np
from scipy.sparse import csr_matrix, coo_matrix
from scipy.optimize import minimize
import copy
from mindquantum import *
import mindspore as ms
import numpy as np
from mindspore import context, nn
from mindspore.ops import operations
from mindquantum import *
from mindspore.nn import Adam, TrainOneStepCell, LossBase
from mindspore.common.initializer import initializer
from mindspore.common.parameter import Parameter
ms.context.set_context(mode=ms.context.PYNATIVE_MODE, device_target="CPU")
import copy

gamma = 0.01
sample_num = 100
fid = 0
for i in range(sample_num):
    real_circ = Circuit()
    real_circ += RY(0.035).on(0)
    real_circ += AmplitudeDampingChannel(gamma).on(0)
    real_circ += RZ(2.861).on(0)
    real_circ += AmplitudeDampingChannel(gamma).on(0)
    real_circ += RY(0.606).on(0)
    real_circ += AmplitudeDampingChannel(gamma).on(0)
    real_circ += RY(0.361).on(1)
    real_circ += AmplitudeDampingChannel(gamma).on(1)
    real_circ += RZ(6.174).on(1)
    real_circ += AmplitudeDampingChannel(gamma).on(1)
    real_circ += RY(4.513).on(1)
    real_circ += AmplitudeDampingChannel(gamma).on(1)
    real_circ += X.on(1,0)
    real_circ += AmplitudeDampingChannel(gamma).on(0)
    real_circ += AmplitudeDampingChannel(gamma).on(1)

    gen_circ = Circuit()
    gen_circ += RY('theta_0').on(2)
    gen_circ += AmplitudeDampingChannel(gamma).on(2)
    gen_circ += RZ('theta_1').on(2)
    gen_circ += AmplitudeDampingChannel(gamma).on(2)
    gen_circ += RY('theta_2').on(2)
    gen_circ += AmplitudeDampingChannel(gamma).on(2)
    gen_circ += RY('theta_3').on(3)
    gen_circ += AmplitudeDampingChannel(gamma).on(3)
    gen_circ += RZ('theta_4').on(3)
    gen_circ += AmplitudeDampingChannel(gamma).on(3)
    gen_circ += RY('theta_5').on(3)
    gen_circ += AmplitudeDampingChannel(gamma).on(3)
    gen_circ += X.on(3,2)
    gen_circ += AmplitudeDampingChannel(gamma).on(3)
    gen_circ += AmplitudeDampingChannel(gamma).on(2)


    disc_circ = Circuit()
    disc_circ += X.on(2, 0)
    disc_circ += AmplitudeDampingChannel(gamma).on(0)
    disc_circ += AmplitudeDampingChannel(gamma).on(2)
    disc_circ += X.on(3, 1)
    disc_circ += AmplitudeDampingChannel(gamma).on(1)
    disc_circ += AmplitudeDampingChannel(gamma).on(3)
    disc_circ += H.on(0)
    disc_circ += AmplitudeDampingChannel(gamma).on(0)
    disc_circ += H.on(1)
    disc_circ += AmplitudeDampingChannel(gamma).on(1)

    ansatz = sum([real_circ, gen_circ, disc_circ])

    index = [int(i, 2) for i in ['0111', '1011', '1101', '1110']]
    m_coo = [coo_matrix(([1], ([i], [i])), shape=(1 << 4, 1 << 4)) for i in index] # << 移位运算
    hams = [Hamiltonian(m.tocsr()) for m in m_coo]

    class my_simulator: 
        def get_expectation_with_grad(self, hams, circ): # 输入为 hams, circ_right, simulator_left 等，这里就只用 hams 代替。
            sim = Simulator('mqvector', 4) 
            def grad_ops(inputs): # 输入为各量子门的参数
                h = 1e-4
                grad = []
                sim.reset()
                sim.apply_circuit(circ, inputs)
                exceptation = [sim.get_expectation(ham).real for ham in hams]
                for i in range(inputs.size):
                    params_p, params_n = copy.deepcopy(inputs), copy.deepcopy(inputs)
                    params_p[i] += h
                    params_n[i] -= h
                    sim.reset()
                    sim.apply_circuit(circ, params_p)
                    g_p = [sim.get_expectation(ham) for ham in hams]
                    sim.reset()
                    sim.apply_circuit(circ, params_n)
                    g_n = [sim.get_expectation(ham) for ham in hams]
                    grad.append([(p.real - n.real)/(2*h) for p, n in zip(g_p, g_n)])
                    sim.reset()
                return exceptation, np.array(grad).T # 运行结果为期望值和梯度
            return grad_ops # MindQuantum 中，这里返回的是一个封装了 hams, grad_ops 等的一个封装器，这里简化为单独的 grad_ops

    class ansatz_only_ops(nn.Cell):
        def __init__(self, expectation_with_grad):
            super().__init__()
            self.expectation_with_grad = expectation_with_grad
            self.shape_ops = operations.Shape()
            self.g = None 

        def construct(self, arg):
            fval, g_ans = self.expectation_with_grad(arg.asnumpy())
            self.g = np.real(g_ans)
            return ms.Tensor(np.real(fval), dtype=ms.float32)

        def bprop(self, arg, out, dout): 
            dout = dout.asnumpy()
            grad = dout @ self.g
            return ms.Tensor(grad, dtype=ms.float32)

    class ansatz_only_layer(nn.Cell):
        def __init__(self, expectation_with_grad, weight='ones'):
            super().__init__()
            self.evolution = ansatz_only_ops(expectation_with_grad)
            weight_size = len(ansatz.params_name)
            self.weight = Parameter(initializer(weight, weight_size, dtype=ms.float32), name='ansatz_weight')

        def construct(self):
            return self.evolution(self.weight)

    class MyLoss(LossBase):
        def __init__(self, reduction='mean'):
            super(MyLoss, self).__init__(reduction)

        def construct(self, logits):
            out = np.sum([logits[i] for i in range(len(logits))])
            return self.get_loss(out)

    class MyWithLossCell(nn.Cell):
        def __init__(self, backbone, loss_fn):
           super(MyWithLossCell, self).__init__(auto_prefix=False)
           self._backbone = backbone
           self._loss_fn = loss_fn

        def construct(self):
           out = self._backbone()
           return self._loss_fn(out)

        @property
        def backbone_network(self):
           return self._backbone

    my_sim = my_simulator()
    grad_ops = my_sim.get_expectation_with_grad(hams, ansatz)

    qnet = ansatz_only_layer(grad_ops)
    loss = MyLoss()
    net_with_criterion = MyWithLossCell(qnet, loss)
    opti = Adam(qnet.trainable_params(), learning_rate=0.05) 
    net = TrainOneStepCell(net_with_criterion, opti)

    for i in range(100):
        res = net()
        
    gen_circ = Circuit()
    gen_circ += RY('theta_0').on(0)
    gen_circ += RZ('theta_1').on(0)
    gen_circ += RY('theta_2').on(0)
    gen_circ += RY('theta_3').on(1)
    gen_circ += RZ('theta_4').on(1)
    gen_circ += RY('theta_5').on(1)
    gen_circ += X.on(1,0)

    state = gen_circ.get_qs(pr=dict(zip(gen_circ.params_name, qnet.weight.asnumpy())))
    state = np.mat(state).reshape((4,1))

    real_circ = Circuit()
    real_circ += RY(0.035).on(0)
    real_circ += RZ(2.861).on(0)
    real_circ += RY(0.606).on(0)
    real_circ += RY(0.361).on(1)
    real_circ += RZ(6.174).on(1)
    real_circ += RY(4.513).on(1)
    real_circ += X.on(1,0)

    target_state = real_circ.get_qs()
    target_state = np.mat(target_state.reshape((4,1)))
    fid += (np.abs(target_state.H@state)**2)[0,0].real
    
fid_mean = fid/sample_num
print(fid_mean)

0.9902105379914649


# 主程序 1：训练 QGAN 的鉴别器。

In [142]:
import numpy as np
from scipy.sparse import csr_matrix, coo_matrix
from scipy.optimize import minimize

from mindquantum import *

gamma = 0.01

real_circ = Circuit()
real_circ += RY(0.035).on(0)
real_circ += AmplitudeDampingChannel(gamma).on(0)
real_circ += RZ(2.861).on(0)
real_circ += AmplitudeDampingChannel(gamma).on(0)
real_circ += RY(0.606).on(0)
real_circ += AmplitudeDampingChannel(gamma).on(0)
real_circ += RY(0.361).on(1)
real_circ += AmplitudeDampingChannel(gamma).on(1)
real_circ += RZ(6.174).on(1)
real_circ += AmplitudeDampingChannel(gamma).on(1)
real_circ += RY(4.513).on(1)
real_circ += AmplitudeDampingChannel(gamma).on(1)
real_circ += X.on(1,0)
real_circ += AmplitudeDampingChannel(gamma).on(0)
real_circ += AmplitudeDampingChannel(gamma).on(1)

gen_circ = Circuit()
gen_circ += RY(0.035).on(2)
gen_circ += AmplitudeDampingChannel(gamma).on(2)
gen_circ += RZ(2.861).on(2)
gen_circ += AmplitudeDampingChannel(gamma).on(2)
gen_circ += RY(0.606).on(2)
gen_circ += AmplitudeDampingChannel(gamma).on(2)
gen_circ += RY(0.361).on(3)
gen_circ += AmplitudeDampingChannel(gamma).on(3)
gen_circ += RZ(6.174).on(3)
gen_circ += AmplitudeDampingChannel(gamma).on(3)
gen_circ += RY(4.513).on(3)
gen_circ += AmplitudeDampingChannel(gamma).on(3)
gen_circ += X.on(3,2)
gen_circ += AmplitudeDampingChannel(gamma).on(2)
gen_circ += AmplitudeDampingChannel(gamma).on(3)


disc_circ = Circuit()
disc_circ += H.on(0)
disc_circ += AmplitudeDampingChannel(gamma).on(0)
disc_circ += Z.on(2, 0)
disc_circ += AmplitudeDampingChannel(gamma).on(0)
disc_circ += AmplitudeDampingChannel(gamma).on(2)
disc_circ += RZ('theta_0').on(0)
disc_circ += AmplitudeDampingChannel(gamma).on(0)
disc_circ += RZ('theta_1').on(2)
disc_circ += AmplitudeDampingChannel(gamma).on(2)
disc_circ += H.on(0)
disc_circ += AmplitudeDampingChannel(gamma).on(0)
disc_circ += H.on(2)
disc_circ += AmplitudeDampingChannel(gamma).on(2)

disc_circ += H.on(1)
disc_circ += AmplitudeDampingChannel(gamma).on(1)
disc_circ += Z.on(3, 1)
disc_circ += AmplitudeDampingChannel(gamma).on(1)
disc_circ += AmplitudeDampingChannel(gamma).on(3)
disc_circ += RZ('theta_2').on(1)
disc_circ += AmplitudeDampingChannel(gamma).on(1)
disc_circ += RZ('theta_3').on(3)
disc_circ += AmplitudeDampingChannel(gamma).on(3)
disc_circ += H.on(1)
disc_circ += AmplitudeDampingChannel(gamma).on(1)
disc_circ += H.on(3)
disc_circ += AmplitudeDampingChannel(gamma).on(3)

ansatz = sum([real_circ, gen_circ, disc_circ])

index = [int(i, 2) for i in ['0111', '1011', '1101', '1110']]
m_coo = [coo_matrix(([1], ([i], [i])), shape=(1 << 4, 1 << 4)) for i in index] # << 移位运算
hams = [Hamiltonian(m.tocsr()) for m in m_coo]
sim = Simulator('mqvector', 4)

def fun(x):
    sim.reset()
    sim.apply_circuit(ansatz, x)
    f = np.sum([sim.get_expectation(ham).real for ham in hams])
    return f

init = [1 for i in range(len(ansatz.params_name))]
res = minimize(fun, init, method='bfgs')
print('期望值：', res.fun)
disc_paras = res.x
print('disc_paras:', disc_paras)

期望值： 1.8888256156876407e-06
disc_paras: [ 0.08427016  0.13703924 -0.15961514 -0.02310518]


# 主程序 2：根据 `主程序 1` 获得的鉴别器参数，训练 QGAN 的生成器

In [152]:
import numpy as np
from scipy.sparse import csr_matrix, coo_matrix
from scipy.optimize import minimize

from mindquantum import *

gamma = 0.01

real_circ = Circuit()
real_circ += RY(0.035).on(0)
real_circ += AmplitudeDampingChannel(gamma).on(0)
real_circ += RZ(2.861).on(0)
real_circ += AmplitudeDampingChannel(gamma).on(0)
real_circ += RY(0.606).on(0)
real_circ += AmplitudeDampingChannel(gamma).on(0)
real_circ += RY(0.361).on(1)
real_circ += AmplitudeDampingChannel(gamma).on(1)
real_circ += RZ(6.174).on(1)
real_circ += AmplitudeDampingChannel(gamma).on(1)
real_circ += RY(4.513).on(1)
real_circ += AmplitudeDampingChannel(gamma).on(1)
real_circ += X.on(1,0)
real_circ += AmplitudeDampingChannel(gamma).on(0)
real_circ += AmplitudeDampingChannel(gamma).on(1)

gen_circ = Circuit()
gen_circ += RY('theta_0').on(2)
gen_circ += AmplitudeDampingChannel(gamma).on(2)
gen_circ += RZ('theta_1').on(2)
gen_circ += AmplitudeDampingChannel(gamma).on(2)
gen_circ += RY('theta_2').on(2)
gen_circ += AmplitudeDampingChannel(gamma).on(2)
gen_circ += RY('theta_3').on(3)
gen_circ += AmplitudeDampingChannel(gamma).on(3)
gen_circ += RZ('theta_4').on(3)
gen_circ += AmplitudeDampingChannel(gamma).on(3)
gen_circ += RY('theta_5').on(3)
gen_circ += AmplitudeDampingChannel(gamma).on(3)
gen_circ += X.on(3,2)
gen_circ += AmplitudeDampingChannel(gamma).on(2)
gen_circ += AmplitudeDampingChannel(gamma).on(3)


disc_circ = Circuit()
disc_circ += H.on(0)
disc_circ += AmplitudeDampingChannel(gamma).on(0)
disc_circ += Z.on(2, 0)
disc_circ += AmplitudeDampingChannel(gamma).on(0)
disc_circ += AmplitudeDampingChannel(gamma).on(2)
disc_circ += RZ(disc_paras[0]).on(0)
disc_circ += AmplitudeDampingChannel(gamma).on(0)
disc_circ += RZ(disc_paras[1]).on(2)
disc_circ += AmplitudeDampingChannel(gamma).on(2)
disc_circ += H.on(0)
disc_circ += AmplitudeDampingChannel(gamma).on(0)
disc_circ += H.on(2)
disc_circ += AmplitudeDampingChannel(gamma).on(2)

disc_circ += H.on(1)
disc_circ += AmplitudeDampingChannel(gamma).on(1)
disc_circ += Z.on(3, 1)
disc_circ += AmplitudeDampingChannel(gamma).on(1)
disc_circ += AmplitudeDampingChannel(gamma).on(3)
disc_circ += RZ(disc_paras[2]).on(1)
disc_circ += AmplitudeDampingChannel(gamma).on(1)
disc_circ += RZ(disc_paras[3]).on(3)
disc_circ += AmplitudeDampingChannel(gamma).on(3)
disc_circ += H.on(1)
disc_circ += AmplitudeDampingChannel(gamma).on(1)
disc_circ += H.on(3)
disc_circ += AmplitudeDampingChannel(gamma).on(3)

ansatz = sum([real_circ, gen_circ, disc_circ])

index = [int(i, 2) for i in ['0111', '1011', '1101', '1110']]
m_coo = [coo_matrix(([1], ([i], [i])), shape=(1 << 4, 1 << 4)) for i in index] # << 移位运算
hams = [Hamiltonian(m.tocsr()) for m in m_coo]
sim = Simulator('mqvector', 4)

def fun(x):
    sim.reset()
    sim.apply_circuit(ansatz, x)
    f = np.sum([sim.get_expectation(ham).real for ham in hams])
    return f

init = [1 for i in range(len(ansatz.params_name))]
res = minimize(fun, init, method='bfgs')
print('期望值为：', res.fun)
print('生成器参数为：', res.x)

real_circ = Circuit()
real_circ += RY(0.035).on(0)
real_circ += RZ(2.861).on(0)
real_circ += RY(0.606).on(0)
real_circ += RY(0.361).on(1)
real_circ += RZ(6.174).on(1)
real_circ += RY(4.513).on(1)
real_circ += X.on(1,0)

gen_circ = Circuit()
gen_circ += RY('theta_0').on(0)
gen_circ += RZ('theta_1').on(0)
gen_circ += RY('theta_2').on(0)
gen_circ += RY('theta_3').on(1)
gen_circ += RZ('theta_4').on(1)
gen_circ += RY('theta_5').on(1)
gen_circ += X.on(1,0)

state = gen_circ.get_qs(pr=dict(zip(gen_circ.params_name, res.x)))
state = np.mat(state).reshape((4,1))

target_state = real_circ.get_qs()
target_state = np.mat(target_state.reshape((4,1)))
print('保真度为：', (np.abs(target_state.H@state)**2)[0,0].real)


期望值为： 1.7301985270152803e-06
生成器参数为： [-0.65835516  3.15744377 -0.07791132  3.02851366  3.48606626  1.61923996]
保真度为： 0.9999783644289637
